# Naive Bayes 분류기로 뉴스 데이터 분류하기
- 출처: [WikiDocs](https://wikidocs.net/22892)

## 1. 뉴스 데이터에 대한 이해
- Scikit-Learn 에서 제공
- Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 18,846개의 뉴스 데이터를 제공
- 훈련 데이터(뉴스 11,314개)와 테스트 데이터(뉴스 7,532개)가 분류되어 있음
- 5개의 속성, 20개의 'target' 속성

In [1]:
# 훈련 데이터 다운로드
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')  # 'train', 'test', 'all'
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [2]:
len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target)

(11314, 11314, 20, 11314)

In [3]:
# 20개의 카테고리 이름
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
# 첫번째 훈련용 뉴스 확인
print(newsdata.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [5]:
# 첫번째 훈련용 뉴스의 카테고리 : 7 - rec.autos
print(newsdata.target[0])

7


## 2. Naive Bayes 분류

### 데이터 전처리
- 뉴스 데이터를 토큰화해주어야 함
- 나이브 베이즈 분류를 위해서는 BoW(Bag of Words)로 만들어주어야 함

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB  # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score     # 정확도 계산

In [7]:
# DTM - Document Term Matrix
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
X_train_dtm.shape
# 훈련용 뉴스의 개수(DTM 관점에서는 문서의 수), 전체 훈련 데이터에 등장한 단어의 수

(11314, 130107)

### 나이브 베이즈 분류 수행

In [8]:
# 사이킷 런에서 제공하는 나이브 베이즈 모델
model = MultinomialNB()
model.fit(X_train_dtm, newsdata.target)
# alpha=1.0은 라플라스 스무딩(확률이 0이 되는 것을 방지)이 적용되었음을 의미

MultinomialNB()

### 테스트 데이터 셋에 대해 정확도 측정

In [9]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) # 테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data)  # 테스트 데이터를 DTM으로 변환

In [10]:
predicted = model.predict(X_test_dtm) #테스트 데이터에 대한 예측
print("정확도: %.4f" % accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7728


### TF-IDF 적용
- DTM 행렬 대신 TF-IDF(Term Frequency - Inverse Document Frequency) 가중치를 적용한 TF-IDF 행렬을 입력으로 텍스트 분류를 수행하면, 성능의 개선을 얻을 수도 있음
- TF-IDF 행렬이 항상 DTM으로 수행했을 때보다 성능이 뛰어나지는 않음

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [13]:
model2 = MultinomialNB()
model2.fit(tfidfv, newsdata.target)

MultinomialNB()

In [14]:
tfidfv_test = tfidf_transformer.transform(X_test_dtm) # DTM을 TF-IDF 행렬로 변환

In [15]:
predicted = model2.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도: %.4f" % accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7739
